# Optimized multiplication of triangular matrices with the NAG Library for Python

In the NAG blog post [Exploting Matrix Structure in the solution of linear systems](https://www.nag.com/content/exploiting-matrix-structure-solution-linear-systems) there is a demonstation showing how paying attention to matrix structure can reap dividends in optimization of computation time.  Using specialised solvers at the right time can significantly speed up your computational workflow.

One case that was of interest to one of our clients was matrix-matrix multiplication where **both** input matrices were triangular.  The standard BLAS specification has a routine called **dtrmm** (also implemented in the NAG Library for Python) which contains optimizations when **one** of the input matrices is triangular but there was nothing that took advantage of the situation where **both** input matrices are triangular.

Two of the new routines in Mark 27 of the NAG library considers the case of matrix-matrix multiplication when both input matrices are [upper or lower triangular](https://en.wikipedia.org/wiki/Triangular_matrix) in structure. The two new routines, **real_tri_matmul_inplace** and **complex_tri_matmul_inplace** implement real and complex versions of this calculation respectively.  

We have attempted to make the new routines as 'BLAS-like' as possible so if you are used to using **dtrmm**, you'll have no problem using the new material. 

In this demonstration, we show how calling this routine from the [NAG Library for Python](https://www.nag.com/nag-library-python) can result in faster multiplications of large matrices compared to using the default matrix-matrix-multiplication or the more specialised **dtrmm** even when an optimized [BLAS](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms) such as the [Intel MKL](https://software.intel.com/en-us/mkl) is in use.

# How to use real_tri_matmul_inplace

Let's start with a simple demonstration showing how to use the routine.  Ths is based on the example in the [Fortran Documentation](https://www.nag.com/numeric/nl/nagdoc_latest/flhtml/f01/f01dgf.html) which computes the matrix product 

$$C = \alpha A^T B$$

where both A and B are upper triangular matrices.

In [1]:
from naginterfaces.library.matop import real_tri_matmul_inplace
import numpy as np

A = np.array([
    [1.5,2.3,6.7,1.9],
    [0.0,3.4,5.4,8.6],
    [0.0,0.0,8.1,2.0],
    [0.0,0.0,0.0,5.9]]
)
    
B = np.array([
    [3.5,2.1,4.0,2.1],
    [0.0,5.6,2.1,2.5],
    [0.0,0.0,1.7,1.1],
    [0.0,0.0,0.0,7.4]]
)

# Is B operated on from the left 'L' or right 'R'
side   = 'L'
# Are A and B upper triangular 'U' or lower triangular 'L'
uplo   = 'U' 
# Are we dealing with A (no transpose) 'N' or transpose(A) 'T'
transa = 'T'
# What's the value of the scalar alpha
alpha = 0.4

In [2]:
real_tri_matmul_inplace(side, uplo, transa, alpha, A, B)

This routine is an **inplace** function which means that there is no return value.  Instead, the input argument **B** is modified in-place to contain the result.

In [3]:
# B Now contains the result of the calculation alpha*transpose(A)*B
B

array([[ 2.1  ,  1.26 ,  2.4  ,  1.26 ],
       [ 3.22 ,  9.548,  6.536,  5.332],
       [ 9.38 , 17.724, 20.764, 14.592],
       [ 2.66 , 20.86 , 11.624, 28.54 ]])

The standard Python equivalent for this calculation is the following although it does not attempt to take advantage of matrix structure at all

In [4]:
A = np.array([
    [1.5,2.3,6.7,1.9],
    [0.0,3.4,5.4,8.6],
    [0.0,0.0,8.1,2.0],
    [0.0,0.0,0.0,5.9]]
)
    
# Redefine B because real_tri_matmul_inplace modified it
B = np.array([
    [3.5,2.1,4.0,2.1],
    [0.0,5.6,2.1,2.5],
    [0.0,0.0,1.7,1.1],
    [0.0,0.0,0.0,7.4]]
)

alpha*(np.transpose(A) @ B)

array([[ 2.1  ,  1.26 ,  2.4  ,  1.26 ],
       [ 3.22 ,  9.548,  6.536,  5.332],
       [ 9.38 , 17.724, 20.764, 14.592],
       [ 2.66 , 20.86 , 11.624, 28.54 ]])

# Speed benefits for large matrices

Matrix-Matrix multiplication is one of the most important operations in computational science. So much so that CPU and GPU manufacturers dedicate some of their hardware to the optimization of this problem.  NVIDIA developed [Tensor Cores](https://www.nvidia.com/en-gb/data-center/tensorcore/) for example and relatively recent advances in CPU instructions such as [AVX (Advanced Vector Extensions)](https://en.wikipedia.org/wiki/Advanced_Vector_Extensions) and [FMA (Fused Multiply-Add)](https://en.wikipedia.org/wiki/FMA_instruction_set) have allowed computers to perform matrix-matrix multiply faster than ever.

Given its importance, matrix-matrix multiplication is heavily optimized in modern software libraries.  In this demonstration, a version of Python's Numpy was used that is linked to [Intel's MKL](https://software.intel.com/en-us/mkl) so the standard matrix multiplication operator works as fast as possible on Intel hardware.

Given triangular matrices and NAG's new routine, we can now do even better!

In [5]:
#generate random upper triangular matrices of size N x N
N = 10000
# For reproducibility
np.random.seed(1)
# Construct random upper triangular matrices
A = np.triu(np.random.random((N, N)))
B = np.triu(np.random.random((N, N)))

# Is B operated on from the left 'L' or right 'R'
side   = 'L'
# Are A and B upper triangular 'U' or lower triangular 'L'
uplo   = 'U' 
# Are we dealing with A (no transpose) 'N' or transpose(A) 'T'
transa = 'N'
# What's the value of the scalar alpha
alpha = 1.0

First, let's see how long this takes using the standard Python operator.  This time, I have modifed the **transa** and **alpha** arguments of NAG routine so that we are simply performing a standard matrix-matrix multiply $C=AB$

In [6]:
%%timeit N = 10000;np.random.seed(1);A = np.triu(np.random.random((N,N)));B = np.triu(np.random.random((N,N)))
#The line of code above is the set-up for each run of timeit. It is not timed. Only the body below is timed
C = A @ B

7.07 s ± 485 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The BLAS routine **dtrmm** includes optimizations for when **one** of the input matrices is triangular.  If the second is also triangular, it won't take advantage of this at all.  It's still useful here though and shows a decent speed-up over the standard matrix-matrix multiplication.

In [7]:
from naginterfaces.library.blas import dtrmm

In [8]:
%%timeit N = 10000;np.random.seed(1);A = np.triu(np.random.random((N,N)));B = np.triu(np.random.random((N,N)));
C = dtrmm(side='L', uplo='U', transa='N', diag='N', m=N, alpha=alpha, a=A, b=B)

3.57 s ± 32.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Finally, let's try the new NAG routine that has no equivalent in BLAS

In [9]:
%%timeit N = 10000;np.random.seed(1);A = np.triu(np.random.random((N,N)));B = np.triu(np.random.random((N,N)))
#The line of code above is the set-up for each run of timeit. It is not timed. Only the body below is timed
#The set up is necessary here because each call to real_tri_matmul_inplace modifies the input matrix B
real_tri_matmul_inplace(side, uplo, transa, alpha, A, B)

2.2 s ± 72.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# BLAS and LAPACK details

When comparing linear algebra routines, it is very important to give details of the BLAS and LAPACK implementations used along with some system details.

In [10]:
# Numpy config details. I used a version of Numpy linked to the Intel MKL and provided by the Intel Python distribution.
np.show_config()

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]


In [11]:
# NAG Library details
from naginterfaces.library.info import impl_details
impl_details()

{'title': 'Linux, 64-bit, Intel C/C++ or Intel Fortran (custom MKL)',
 'integer_bits': 64,
 'precision': 'double',
 'product_code': 'NLL6I27DVL',
 'mark': (27, 0, 2),
 'vendor_lib': 'MKL 2019.0.3',
 'build_id': 202348,
 'hardware': 'x86_64',
 'os': 'Linux 2.6.32-696.6.3.el6.x86_64',
 'compilers': {'C': {'title': 'Intel icc Version 19.0.3.199 20190206',
   'flags': '-O3 -axCORE-AVX2,AVX -no-fma -fp-model precise -fp-speculation=safe -fPIC -m64 -w -fexceptions'},
  'C++': {'title': 'Intel icpc Version 19.0.3.199 20190206',
   'flags': '-O3 -axCORE-AVX2,AVX -no-fma -fp-model precise -fp-speculation=safe -fPIC -m64 -w -fexceptions'},
  'Fortran': {'title': 'Intel ifort Version 19.0.3.199 20190206',
   'flags': '-O3 -axCORE-AVX2,AVX -no-fma -fp-model precise -fp-speculation=safe -auto -fPIC -threads -w -fexceptions -i8'}}}

In [12]:
# Operating system and CPU details
import platform
import psutil
print(platform.architecture())
print(platform.processor())
print("Number of Physical CPUs = {0}".format(psutil.cpu_count(logical=False)))
print("Number of Logical CPUs = {0}".format(psutil.cpu_count(logical=True)))
      

('64bit', 'ELF')
x86_64
Number of Physical CPUs = 8
Number of Logical CPUs = 16
